In [ ]:

from utils.utils import get_documents_from_index, convert_text_to_ascii
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
from pydantic import BaseModel, Field
from typing import Literal
# from ..globals import *
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from typing import Literal
import os


In [ ]:


# Workflow constants
NUM_FOLLOWUP_Q = 3
TOP_N_DOCUMENTS = 5

# LLM Stuff
API_KEY = os.getenv("OPENAI_KEY")
AZURE_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
API_VERSION = "2025-01-01-preview"


compare_questions_human = """
#USER_QUESTION: {user_query}

#index_query_1: {index_query_1}
#index_query_2: {index_query_2}
#index_query_3: {index_query_3}
#index_query_4: {index_query_4}
#index_query_5: {index_query_5}

"""
prompt = """Your task is to evaluate the semantic similarity between a given user query (which may contain multiple parts or compound questions) and a list of indexed questions. The goal is to determine whether the meaning and intent of the user query and each indexed question are the same or different within the context of the medical domain for UnitedHealth Group.  
  
### **Independent Evaluation:**  
- Assess each indexed question independently.  
- Do not let the content or context of previous indexed questions influence your evaluation of the current question.  
- Treat each indexed question as a standalone comparison with the user query.  
- **Compound/Multiple-Part Queries**:   
  - If the user query contains multiple parts, the match is only **True** if the indexed question aligns with **all parts** of the user query.   
  - If any part of the user query does not align with the indexed question, the match must be **False**.  
  
### **Key Requirements:**  
- **Semantic Similarity in the Medical Domain**:  
  - Evaluate alignment based on meaning and intent, even if the wording differs.  
  - Pay close attention to healthcare-specific differences in terminology, such as "coverage," "cost," "copay," "coinsurance," "provider," and "specialist."  
  - Ensure that the **context** of the medical domain remains consistent across all parts of the user query and indexed question.  
  - For compound queries, the intent of **every part** of the user query must match the indexed question for a **True** result.   
  - If an indexed question addresses only **one part** of a compound query while ignoring other parts, the match must be **False**.  
  
### **Decision-Making Criteria:**  
- **True**: The user query and indexed question express the same meaning and intent across **all parts** of the user query (terminologies match, context matches, and specific focuses align for every part).  
- **False**: The user query and indexed question express different meanings or intents, **or if any part of a compound query does not align** with the indexed question.  
  
### **Examples of Common Scenarios:**  
1. **Exact Match of All Parts**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question: "Is eyewear covered under the plan and does it cover home health care?"  
   - **True** (Both parts match the intent and meaning exactly).  
  
2. **Partial Match for Compound Questions**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question 1: "Is eyewear covered under this plan?"  
   - Indexed Question 2: "Does this plan cover home health care?"  
   - **False** (Each indexed question addresses only one part of the compound query and ignores the other part, making it a partial match).  
  
3. **Different Focus**:  
   - User Query: "Is eyewear covered under the plan and does it cover home health care?"  
   - Indexed Question: "Does this plan cover general eye exams?"  
   - **False** (The user query focuses on eyewear coverage, while the indexed question focuses on eye exams, leading to different intents).  
  
4. **Different Terminology in Compound Query**:  
   - User Query: "Can I see a provider and is the copay affordable?"  
   - Indexed Question: "Can I see a provider?"  
   - **False** (The first part aligns, but the second part about copay affordability does not match).  
  
**Handling Ambiguity:**  
- If a question is ambiguous or unclear, use the context of the medical domain and the intent of the user query to make a reasoned decision.  
- If intent cannot be reasonably inferred, default to False and provide a clear explanation.  
- The user query and indexed question have partial match it should be False.
  
**Output Format:**  
For each indexed question, provide a True or False response along with reasoning in JSON format. Ensure that your evaluation for each question is independent.   
  
Example output:  
```json  
{  
    "is_similar1": "False",  
    "reason1": "The user query is about seeing doctors out of network, while the indexed question is unrelated.",  
    "is_similar2": "True",  
    "reason2": "The user query aligns with the coverage question for out of network doctors.",  
    "is_similar3": "False",  
    "reason3": "The user query is about seeing out of network providers and eye cost, which is partial match.",  
    "is_similar4": "False",  
    "reason4": "The user query addresses the ability to see a doctor, while the indexed question is about specific costs.",  
    "is_similar5": "False",  
    "reason5": "The user query is about general doctors, while the indexed question specifies a specialist."  
} ``` """

from typing import Literal
class SimilarityData(BaseModel):  
    is_similar1: Literal[True, False]  
    is_similar2: Literal[True, False]  
    is_similar3: Literal[True, False]  
    is_similar4: Literal[True, False]  
    is_similar5: Literal[True, False]  
    Reason1: str
    Reason2: str
    Reason3: str
    Reason4: str
    Reason5: str

	  
compare_questions_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(prompt),
        HumanMessagePromptTemplate.from_template(
            template=compare_questions_human,
            input_variables=["user_query", "index_query_1", "index_query_2", "index_query_3", "index_query_4", "index_query_5"]
        ),
    ]
)

	  
llm = AzureChatOpenAI(
    api_key=API_KEY,  # type:ignore
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o-mini",
    api_version=API_VERSION,
    temperature=0,
)

compare_questions_chain = compare_questions_template | llm.with_structured_output(SimilarityData, method="json_schema")


async def async_compare_questions(user_query: str, index_query_1: str, index_query_2: str, index_query_3: str, index_query_4: str, index_query_5: str):
    print("user_query:" + user_query)
    print("index_query_1: " + index_query_1)
    print("index_query_2: " + index_query_2)
    print("index_query_3: " + index_query_3)
    print("index_query_4: " + index_query_4)
    print("index_query_5: " + index_query_5)
    """
    Asynchronously compares two questions using the compare_questions_chain.

    Args:
        user_query (str): The user's query to be compared.
        index_query (str): The index query to compare against.

    Returns:
        tuple: A tuple containing the user query, index query, and the response from the comparison.
    """

    response = await compare_questions_chain.ainvoke(
        input={"user_query": user_query,
                "index_query_1": index_query_1,
                "index_query_2": index_query_2, 
                "index_query_3": index_query_3, 
               "index_query_4": index_query_4, 
               "index_query_5": index_query_5}
    )

    return response
  
	  



In [ ]:

user_query = "What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

In [ ]:
user_query = "is eyewear covered under the plan and does it cover home health care? "
index_query_1 =  "Is eyewear covered under this plan?"
index_query_2 = "Does this plan cover home health care?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

In [ ]:
index_query_1 = "What's the cost to see a provider?"

index_query_2 = "What would it cost me to see a doc?"

# index_query_3 = "Provider costs?"
index_query_3 = "What's the copay to see a doctor?"

index_query_5 = "How much would it cost to see a specialist?"

index_query_4 = "What's the coinsurance for PCP visits?"
user_query = "How much does it cost to see a doctor?"

In [ ]:
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
index_query_3 = "Provider costs?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan offer OTC?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Is eyewear covered under this plan?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
index_query_1 =  "Does this plan offer OTC?"

index_query_2 = "Does this plan cover home health care?"

index_query_3 = "Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?"

index_query_4 = "Can I see doctors out of the network?"
index_query_5 = "Is eyewear covered under this plan?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:

user_query = "Can I see providers out of the network?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan offer OTC?"
index_query_4 = "Can I see doctors out of the network?"
index_query_5 = "Can I see an OON specialist?"
index_query_3 = "Are OON docs covered??"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan offer OTC?"
index_query_1 = "Can I see providers outside of the network?"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
# index_query_5 = "What's the coinsurance for out of network PCP visits?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:



user_query = "Can I see doctors out of the network?"
index_query_1 = "Does this plan  offer OTC"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

In [ ]:
user_query = "What are the copayments or coinsurance amounts for my regular doctor visits and hospital stays?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

In [ ]:

user_query = "How much will it cost to see my doctor and what is co-insurance for a hospital visit?"
index_query_1 = "How much will it cost to see my doctor?"
index_query_2 = "How much will it cost if I go to the hospital?"
index_query_3 = "Does this plan offer OTC?"
index_query_4 = "Does this plan cover crutches?"
index_query_5 = "What is my out of pocket maximum?"

response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
for res in response:
    print(res)

In [ ]:
user_query = "How much does it cost to see a doctor?"
index_query_1 = "Can I see providers outside of the network?"
index_query_2 = "Are OON docs covered?"
index_query_3 = "Out of network providers?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
# index_query_5 = "What's the coinsurance for out of network PCP visits?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:

user_query = "Is eyewear covered under this plan?"
index_query_1 = "Does this plan include coverage for eyewear?"
index_query_2 = "Are glasses or contact lenses covered?"
index_query_3 = "Is vision care included in this plan?"
index_query_4 = "What's the copay for out of network doctors?" #diff
index_query_5 = "Can I see an OON specialist?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan offer OTC?"
index_query_1 = "What OTC items are covered under this plan?" #diff
index_query_2 = "How much is the allowance for OTC products?" #diff
index_query_3 = "Can I get OTC medications with this plan?" #diff
index_query_4 = "Does this plan include OTC benefits?"
index_query_5 = "Are over-the-counter items covered?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan cover home health care?"
index_query_1 = "Is home health care included in this plan?"
index_query_2 = "Does this plan provide home health services?"
index_query_3 = "Are home health care services covered?"
index_query_4 = "What types of home health care are covered?" #diff
index_query_5 = "Is there a limit on home health care visits?" #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Does this plan offer special benefits or programs for managing chronic conditions, like diabetes or heart disease?"
index_query_1 = "Are there special programs for chronic conditions in this plan?"
index_query_2 = "What chronic conditions are covered under special programs?" #diff
index_query_3 = "Are there any wellness programs for chronic disease management?" #diff
index_query_4 = "Does this plan include benefits for managing diabetes or heart disease?"
index_query_5 = "Are chronic condition management programs offered?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "Is eyewear covered under this plan and what is copay to visit doctor?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5) 
response

In [ ]:
user_query = "Does this plan cover home health care?"
index_query_1, index_query_2, index_query_3 = 'Is home health care included in this plan?', 'Does this plan provide home health services?', 'Are home health care services covered?'
index_query_4, index_query_5 = 'What types of home health care are covered?', 'Is there a limit on home health care visits?' #diff
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "What is my monthly premium and does it cover hearing aid?"
index_query_1 = "What is my monthly premium?"
index_query_2 = " does it cover hearing aid?"
index_query_3 = "What is my monthly premium and does it cover hearing aid?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response

In [ ]:
user_query = "What is my out of pocket maximum and what is my PCP copay?"
index_query_1 = "what is my PCP copay?"
index_query_2 = "What is my out of pocket maximum and what is my PCP copay?"
index_query_3 = "What is my out of pocket maximum?"
response = await async_compare_questions(user_query, index_query_1, index_query_2, index_query_3, index_query_4, index_query_5)
response